In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

In [5]:
df = pd.read_csv('../data/users.csv')

In [6]:
df.head()

,id,name,email,date_of_birth,sex,weight_value,height_value,activity_level,exercise_frequency,health_conditions,goal_type,motivation
0,bb8fe9cf-19db-42ac-bcee-f216f1d0c6f7,Bob Lee,boblee6068@example.com,1977-12-21,male,99.6,184.9,low,7,"['diabetes', 'hypertension']",['muscle_gain'],Get fit for summer
1,16974410-b71a-45f9-aa8e-acd20b711737,Jane Smith,janesmith297@example.com,1991-06-01,female,98.1,159.8,moderate,6,['high_cholesterol'],['weight_loss'],Doctor recommended
2,02352d6f-d182-466b-9321-bf8f7c604b43,Charlie Kim,charliekim7269@example.com,1998-11-29,non-binary,104.7,185.6,low,4,['hypertension'],['maintenance'],General wellness
3,7af66694-961f-4ac2-8911-d5402d1af23d,Alice Johnson,alicejohnson4877@example.com,1975-12-17,female,70.3,167.3,high,5,['none'],['maintenance'],General wellness
4,5b1a599b-beb6-41ed-8c31-fc641bcf3115,Elliot Alderson,elliotalderson4424@example.com,1988-06-25,female,98.9,192.8,moderate,1,['high_cholesterol'],['maintenance'],Doctor recommended


## BMR Calculations

In [8]:
def mifflin_st_jeor(sex, weight, height, age):
    if sex.lower() == 'male':
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161

def katch_mcardle(weight, body_fat_percent):
    lean_mass = weight * (1 - body_fat_percent / 100)
    return 370 + 21.6 * lean_mass

def weighted_bmr(sex, weight, height, age, body_fat_percent=None):
    bmr_msj = mifflin_st_jeor(sex, weight, height, age)
    if body_fat_percent is not None:
        bmr_kma = katch_mcardle(weight, body_fat_percent)
        return 0.8 * bmr_msj + 0.2 * bmr_kma
    else:
        return bmr_msj


## PAL Multiplier Mapping & TDEE Calculation

In [9]:
# Map daily activity (1-5) + weekly exercise (0-7) to PAL scale
def get_pal(activity_level, exercise_freq):
    if activity_level == "very_low":
        return 1.2
    if activity_level == "low":
        return 1.3 if exercise_freq <= 2 else 1.45
    if activity_level == "moderate":
        return 1.55 if exercise_freq <= 3 else 1.65
    if activity_level == "high":
        return 1.75 if exercise_freq <= 4 else 1.9
    return 2.0  # very high

def calculate_tdee(bmr, activity_level, exercise_freq):
    pal = get_pal(activity_level, exercise_freq)
    return bmr * pal


## Devine Formula for Ideal Body Weight (IBW)

In [10]:
def calculate_ibw(sex, height_cm):
    height_in = height_cm / 2.54
    if sex.lower() == 'male':
        return 50 + 2.3 * (height_in - 60)
    else:
        return 45.5 + 2.3 * (height_in - 60)


## Macronutrient Distribution by Goal

In [11]:
def get_macros(tdee, goal_type, ibw=None, conditions=[], muscle_gain=False):
    if "weight_loss" in goal_type:
        oc = tdee - 500
    elif "weight_gain" in goal_type:
        oc = tdee + 300
    else:  # maintain
        oc = tdee

    # Base macros
    carbs = 0.45 * oc / 4
    protein = 0.30 * oc / 4
    fat = 0.25 * oc / 9

    # Muscle gain override
    if muscle_gain and ibw:
        protein = 2.4 * ibw
        fat = 0.25 * oc / 9
        carbs = (oc - (protein * 4 + fat * 9)) / 4

    # Diabetes override
    if "diabetes" in conditions or "pre-diabetes" in conditions:
        carbs = 0.40 * oc / 4
        fat = (oc - (protein * 4 + carbs * 4)) / 9

    return {
        "optimal_calories": round(oc),
        "carbs_g": round(carbs),
        "protein_g": round(protein),
        "fat_g": round(fat)
    }


In [16]:
def calculate_age(dob):
    dob = pd.to_datetime(dob)
    today = datetime.today()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

def cm_to_inch(cm):
    return cm / 2.54

def kg_to_lb(kg):
    return kg * 2.20462

## Apply to all users 

In [18]:
from datetime import datetime

def get_age(dob):
    dob = datetime.strptime(dob, "%Y-%m-%d")
    return (datetime.today() - dob).days // 365

results = []

for _, row in df.iterrows():
    age = get_age(row['date_of_birth'])
    weight = row['weight_value']
    height = row['height_value']
    sex = row['sex']
    activity = row['activity_level']
    exercise = row['exercise_frequency']
    goals = eval(row['goal_type']) if isinstance(row['goal_type'], str) else []
    conditions = eval(row['health_conditions']) if isinstance(row['health_conditions'], str) else []
    motivation = row['motivation']
    
    # Estimate body fat for now (could be user input later)
    estimated_bf = 20 if sex.lower() == 'male' else 28  # adjustable default

    # Calculate all required values
    bmr = weighted_bmr(sex, weight, height, age, estimated_bf)
    tdee = calculate_tdee(bmr, activity, exercise)
    ibw = calculate_ibw(sex, height)
    macros = get_macros(tdee, goals, ibw, conditions, "muscle_gain" in goals)

    # Append results
    results.append({
        "id": row["id"],
        "name": row["name"],
        "age": age,
        "bmr": round(bmr, 2),
        "tdee": round(tdee, 2),
        "ibw": round(ibw, 2),
        "optimal_calories": macros["optimal_calories"],
        "carbs_g": macros["carbs_g"],
        "protein_g": macros["protein_g"],
        "fat_g": macros["fat_g"],
        "goals": goals,
        "health_conditions": conditions,
        "motivation": motivation
    })

# Create final DataFrame
nutrition_df = pd.DataFrame(results)

# Display first few rows
nutrition_df.head()


,id,name,age,bmr,tdee,ibw,optimal_calories,carbs_g,protein_g,fat_g,goals,health_conditions,motivation
0,bb8fe9cf-19db-42ac-bcee-f216f1d0c6f7,Bob Lee,47,1955.52,2835.50,79.43,2836,284,191,104,[muscle_gain],"[diabetes, hypertension]",Get fit for summer
1,16974410-b71a-45f9-aa8e-acd20b711737,Jane Smith,33,1702.13,2808.51,52.20,2309,260,173,64,[weight_loss],[high_cholesterol],Doctor recommended
2,02352d6f-d182-466b-9321-bf8f7c604b43,Charlie Kim,26,1932.46,2802.07,75.56,2802,315,210,78,[maintenance],[hypertension],General wellness
3,7af66694-961f-4ac2-8911-d5402d1af23d,Alice Johnson,49,1366.76,2596.85,58.99,2597,292,195,72,[maintenance],[none],General wellness
4,5b1a599b-beb6-41ed-8c31-fc641bcf3115,Elliot Alderson,36,1864.02,2889.23,82.08,2889,325,217,80,[maintenance],[high_cholesterol],Doctor recommended
